In [72]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import confusion_matrix

In [77]:
df = pd.read_excel('table_8_offenses_known_to_law_enforcement_new_york_by_city_2013.xls', \
                   skip_rows=[0, 1, 2, 3], header=4, nrows=348)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 13 columns):
City                                    348 non-null object
Population                              348 non-null int64
Violent
crime                           348 non-null int64
Murder and
nonnegligent
manslaughter    348 non-null int64
Rape
(revised
definition)1              0 non-null float64
Rape
(legacy
definition)2               348 non-null int64
Robbery                                 348 non-null int64
Aggravated
assault                      348 non-null int64
Property
crime                          348 non-null int64
Burglary                                348 non-null int64
Larceny-
theft                          348 non-null int64
Motor
vehicle
theft                     348 non-null int64
Arson3                                  187 non-null float64
dtypes: float64(2), int64(10), object(1)
memory usage: 35.4+ KB


In [78]:
df.rename(columns={df.columns[3]: "Murder1"}, inplace=True)
df.rename(columns={df.columns[4]: "Rape1"}, inplace=True)
df.rename(columns={df.columns[5]: "Rape2"}, inplace=True)
df.fillna(0)

,City,Population,Violent crime,Murder1,Rape1,Rape2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3
0,Adams Village,1861,0,0,0.0,0,0,0,12,2,10,0,0.0
1,Addison Town and Village,2577,3,0,0.0,0,0,3,24,3,20,1,0.0
2,Akron Village,2846,3,0,0.0,0,0,3,16,1,15,0,0.0
3,Albany,97956,791,8,0.0,30,227,526,4090,705,3243,142,0.0
4,Albion Village,6388,23,0,0.0,3,4,16,223,53,165,5,0.0
5,Alfred Village,4089,5,0,0.0,0,3,2,46,10,36,0,0.0
6,Allegany Village,1781,3,0,0.0,0,0,3,10,0,10,0,0.0
7,Amherst Town,118296,107,1,0.0,7,31,68,2118,204,1882,32,3.0
8,Amityville Village,9519,9,0,0.0,2,4,3,210,16,188,6,1.0
9,Amsterdam,18182,30,0,0.0,0,12,18,405,99,291,15,0.0


In [79]:
df.head()

,City,Population,Violent crime,Murder1,Rape1,Rape2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3
0,Adams Village,1861,0,0,NaN,0,0,0,12,2,10,0,0.0
1,Addison Town and Village,2577,3,0,NaN,0,0,3,24,3,20,1,0.0
2,Akron Village,2846,3,0,NaN,0,0,3,16,1,15,0,0.0
3,Albany,97956,791,8,NaN,30,227,526,4090,705,3243,142,NaN
4,Albion Village,6388,23,0,NaN,3,4,16,223,53,165,5,NaN


In [84]:
city = pd.get_dummies(df.City)

In [85]:
df.drop(['City'], inplace=True, axis=1)

In [86]:
df_merge = pd.concat([df, city], axis=1)

In [87]:
# Drop the column with NULL values
df_merge.drop(df_merge.columns[3], axis=1, inplace=True)

In [100]:
df_merge.head()

,Population,Violent crime,Murder1,Rape2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,...,Westhampton Beach Village,White Plains,Whitehall Village,Whitesboro Village,Whitestown Town,Woodbury Town,Woodridge Village,Woodstock Town,Yonkers,Yorktown Town
0,1861,0,0,0,0,0,12,2,10,0,...,0,0,0,0,0,0,0,0,0,0
1,2577,3,0,0,0,3,24,3,20,1,...,0,0,0,0,0,0,0,0,0,0
2,2846,3,0,0,0,3,16,1,15,0,...,0,0,0,0,0,0,0,0,0,0
3,97956,791,8,30,227,526,4090,705,3243,142,...,0,0,0,0,0,0,0,0,0,0
4,6388,23,0,3,4,16,223,53,165,5,...,0,0,0,0,0,0,0,0,0,0


In [101]:
df_merge['Murder1'] = pd.to_numeric(df_merge['Murder1'])

In [105]:
df_merge.loc[df_merge.Murder1 > 0, 'Murder1'] = 1
df_merge.loc[df_merge.Robbery > 0, 'Robbery'] = 1

In [106]:
df_merge.head()

,Population,Violent crime,Murder1,Rape2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,...,Westhampton Beach Village,White Plains,Whitehall Village,Whitesboro Village,Whitestown Town,Woodbury Town,Woodridge Village,Woodstock Town,Yonkers,Yorktown Town
0,1861,0,0,0,0,0,12,2,10,0,...,0,0,0,0,0,0,0,0,0,0
1,2577,3,0,0,0,3,24,3,20,1,...,0,0,0,0,0,0,0,0,0,0
2,2846,3,0,0,0,3,16,1,15,0,...,0,0,0,0,0,0,0,0,0,0
3,97956,791,1,30,1,526,4090,705,3243,142,...,0,0,0,0,0,0,0,0,0,0
4,6388,23,0,3,1,16,223,53,165,5,...,0,0,0,0,0,0,0,0,0,0
